In [ ]:
# default_exp inventory

In [ ]:
#hide
!pip install -q nbdev lambdasdk s3bz

In [ ]:
#hide
import pickle, os
KEY = ''
PW = ''
keypath = '/Users/nic/.villa-product-master-inventory'
if KEY and PW:
  with open (keypath, 'wb') as f:
    pickle.dump({
        'KEY': KEY,
        'PW': PW
    }, f)
if os.path.exists(keypath):
  with open(keypath, 'rb') as f:
    creden = pickle.load(f)
else:
  creden = {}
USER = creden.get('KEY')
PW = creden.get('PW')

# Inventory

> upload and download inventory data from villa master backend

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
from json.decoder import JSONDecodeError
from botocore.config import Config
from s3bz.s3bz import S3, Requests
from lambdasdk.lambdasdk import Lambda
from awsSchema.apigateway import Event, Response
import bz2, json, boto3, base64, logging,itertools

In [ ]:
#export
def union(*dicts):
    return dict(itertools.chain.from_iterable(dct.items() for dct in dicts))
class Endpoints:
  '''get endpoint names from branch name'''
  def __init__(self, branchName='manual-dev'):
    self.branchName = branchName
  updateWithS3 = lambda self: f'update-inventory-s3-{self.branchName}'
  inputS3 = lambda self: f'input-bucket-{self.branchName}'
  querySingleProduct = lambda self: f'single-product-query-inventory-{self.branchName}'
  queryAll = lambda self: f'query-all-inventory-{self.branchName}'
  queryBranch = lambda self: f'query-branch-inventory-{self.branchName}'
  
  
class InventorySdk:
  ''' interact with villa inventory database '''
  def __init__(self, branchName = 'dev', user = None, pw = None, 
               region = 'ap-southeast-1'):
    self.branchName = branchName
    self.lambdaClient = Lambda(user =user, pw=pw, region = region)
    self.user = user; self.pw = pw; self.region = region
    self.endpoint = Endpoints(branchName=branchName)
    
    
  def updateWithS3(self, data:dict, 
                   key:str = 'allProducts',
                   invocationType:str = 'Event'):
    
    # save to s3
    S3.save(key = key, 
            objectToSave = data , 
            bucket = self.endpoint.inputS3(),
            user=self.user, pw=self.pw)
    logging.info(f'saving to s3 completed')
    
    lambdaPayload = {
        'inputBucketName': self.endpoint.inputS3(),
        'inputKeyName': key
    }
    logging.info(f'input to lambda is {lambdaPayload}')
    try:
      result = self.lambdaClient.invoke(functionName= self.endpoint.updateWithS3() 
                                        ,input=lambdaPayload,
                                        invocationType= invocationType )
      if result: return Response.getReturn(result)
    except JSONDecodeError:
      logging.warning('no return from function')
      return True

  def querySingleProduct(self, ib_prcode= None, functionName=None, 
                         user=None, pw=None):
    '''query a single product'''
    functionName = functionName or self.endpoint.querySingleProduct()
    input = { "body": json.dumps({'ib_prcode': ib_prcode })}
    response =  self.lambdaClient.invoke(
        functionName = functionName, input = input )
    try:
      inventory = json.loads(Response.from_dict(response).body)
      return {k:union(v,{'ib_prcode':ib_prcode,'ib_brcode':k}) \
              if k.isdigit() else v for k,v in inventory.items()}
    except:
      return response

  def queryAll(self, functionName = None):
    '''get the whole database'''
    functionName = functionName or self.endpoint.queryAll()
    response =  self.lambdaClient.invoke(
        functionName = functionName, input = {} )
    responseBody = json.loads(Response.from_dict(response).body)
    ### return body
    if 'url' in responseBody:
      inventory =  Requests.getContentFromUrl(responseBody['url'])
      return {k:{k2: union(v2, {'ib_prcode':k,'ib_brcode': k2}) if k2.isdigit() 
    else v2 for k2,v2 in v.items()} if k.isdigit() else v for k,v in inventory.items()}
#       return {k:{k2: union(json.loads(v2), {'ib_prcode':k,'ib_brcode': k2})for k2,v2 in v.items()} for k,v in inventory.items()}
    else :
      logging.error(responseBody)
      return responseBody
  
  def queryBranch(self, branch = '1000', functionName = None):
    '''get the branch database'''
    functionName = functionName or self.endpoint.queryBranch()
    response =  self.lambdaClient.invoke(
        functionName = functionName, input = {'body':json.dumps({'branch':branch})} )
    responseBody = json.loads(Response.from_dict(response).body)
    ### return body
    if 'url' in responseBody:
      inventory = Requests.getContentFromUrl(responseBody['url'])
      return {k:union(v,{'ib_prcode':k,'ib_brcode':branch}) for k,v in inventory.items()}
    else :
      logging.error(responseBody)
      return responseBody

In [ ]:
from dataclasses import dataclass
from dataclasses_json import dataclass_json
from random import randrange
from datetime import datetime
import boto3


## generate dummy data for testing

In [ ]:
%%time
#Dummy Data
numberOfRows = 1000
@dataclass_json
@dataclass
class Inventory:
  ib_prcode:str
  ib_brcode:str
  ib_cf_qty:str
  new_ib_vs_stock_cv:str

sampleLargeRandomInput = [ Inventory.from_dict({
    'ib_brcode' : str(randrange(1000,1030,1)),
    'ib_prcode' : str(randrange(10000,100000,1)),
    'ib_cf_qty' : str(randrange(-10,1000,1)),
    'new_ib_vs_stock_cv' : str(randrange(-10,1000,1))
  }).to_dict() for _ in range(numberOfRows)]
sampleLargeRandomInput[0]

CPU times: user 108 ms, sys: 131 µs, total: 108 ms
Wall time: 107 ms


{'ib_prcode': '40340',
 'ib_brcode': '1012',
 'ib_cf_qty': '151',
 'new_ib_vs_stock_cv': '603'}

## create main object

In [ ]:
sdk = InventorySdk(user=USER, pw=PW, branchName = 'dev-manual')

## Upload the batch data using s3

In [ ]:
%%time
print(f'uploading {len(sampleLargeRandomInput)} items')
result = sdk.updateWithS3(
    sampleLargeRandomInput,
    invocationType = 'Event'
  )
result

uploading 1000 items
CPU times: user 57.7 ms, sys: 8.71 ms, total: 66.4 ms
Wall time: 195 ms


{'body': 'true', 'statusCode': 200, 'header': {}}

## Query Single Product

In [ ]:
%%time
sdk.querySingleProduct('0000002')

CPU times: user 14.3 ms, sys: 553 µs, total: 14.9 ms
Wall time: 110 ms


{'ib_prcode': '0000002',
 '1000': {'ib_cf_qty': 35,
  'new_ib_bs_stock_cv': 33,
  'lastUpdate': 1600567810.529301,
  'ib_prcode': '0000002',
  'ib_brcode': '1000'},
 '1001': {'ib_cf_qty': 32,
  'new_ib_bs_stock_cv': 30,
  'lastUpdate': 1600567810.529316,
  'ib_prcode': '0000002',
  'ib_brcode': '1001'},
 '1002': {'ib_cf_qty': 34,
  'new_ib_bs_stock_cv': 30,
  'lastUpdate': 1600567810.529318,
  'ib_prcode': '0000002',
  'ib_brcode': '1002'},
 'lastUpdate': 1600567810.529318}

In [ ]:
%%time
sdk.querySingleProduct('392070')

CPU times: user 3.49 ms, sys: 303 µs, total: 3.8 ms
Wall time: 33.2 ms


{'body': '"missing product {\'ib_prcode\': \'392070\'}"',
 'statusCode': 400,
 'header': {}}

## Query a branch

In [ ]:
%%time
result = sdk.queryBranch('1000')
#showing the first 2 result
list(iter(result.items()))[:2]

CPU times: user 377 ms, sys: 19 ms, total: 397 ms
Wall time: 787 ms


[('0000009',
  {'ib_cf_qty': 50,
   'new_ib_bs_stock_cv': 27,
   'lastUpdate': 1602338504.869655,
   'ib_prcode': '0000009',
   'ib_brcode': '1000'}),
 ('0000002',
  {'ib_cf_qty': 35,
   'new_ib_bs_stock_cv': 33,
   'lastUpdate': 1600567810.529301,
   'ib_prcode': '0000002',
   'ib_brcode': '1000'})]

## Query All

In [ ]:
%%time
result = sdk.queryAll()
list(iter(result.items()))[:2]

CPU times: user 2.3 s, sys: 87.1 ms, total: 2.39 s
Wall time: 2.73 s


[('0000009',
  {'ib_prcode': '0000009',
   '1000': {'ib_cf_qty': 50,
    'new_ib_bs_stock_cv': 27,
    'lastUpdate': 1602338504.869655,
    'ib_prcode': '0000009',
    'ib_brcode': '1000'},
   'lastUpdate': 1602338504.869655}),
 ('0000002',
  {'ib_prcode': '0000002',
   '1000': {'ib_cf_qty': 35,
    'new_ib_bs_stock_cv': 33,
    'lastUpdate': 1600567810.529301,
    'ib_prcode': '0000002',
    'ib_brcode': '1000'},
   '1001': {'ib_cf_qty': 32,
    'new_ib_bs_stock_cv': 30,
    'lastUpdate': 1600567810.529316,
    'ib_prcode': '0000002',
    'ib_brcode': '1001'},
   '1002': {'ib_cf_qty': 34,
    'new_ib_bs_stock_cv': 30,
    'lastUpdate': 1600567810.529318,
    'ib_prcode': '0000002',
    'ib_brcode': '1002'},
   'lastUpdate': 1600567810.529318})]

In [ ]:
#hide
from nbdev.export import *
notebook2script()

Converted index.ipynb.
Converted inventory.ipynb.


## schema
```
key:str # key is ib_prcode
  ib_cf_qty: int
  new_ib_bs_stock_cv: int
  lastUpdate: float
  ib_brcode: str
  ib_prcode: str
```